# Trabajo de machine lerning (Clasificador de SPAM)

## Recogemos los datos

In [1]:
import numpy as np
import pandas as pd
from info import CSV_ROUTE
from aux_functions import conversion_target

dataset = pd.read_csv(CSV_ROUTE, encoding= 'latin-1')

dataset_limpio = dataset[['class', 'message']]

dataset_limpio['target'] = dataset_limpio['class'].apply(conversion_target)

dataset_limpio.head()

/tmp/ipykernel_148677/408128277.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_limpio['target'] = dataset_limpio['class'].apply(conversion_target)


,class,message,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [28]:
dataset_limpio

,class,message,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will Ì_ b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


## Dividimos los datos

In [3]:
from sklearn.model_selection import train_test_split

X = dataset_limpio['message']
y = dataset_limpio['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

## Creamos el Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

naive_bayes = Pipeline(steps = [
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('bayes', MultinomialNB())
])

vectorizer_parameters = {
    'vectorizer__max_features': range(80, 400)
}

bayes_grid = GridSearchCV(naive_bayes,
                         vectorizer_parameters)

bayes_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vectorizer',
                                        CountVectorizer(stop_words='english')),
                                       ('bayes', MultinomialNB())]),
             param_grid={'vectorizer__max_features': range(80, 400)})

In [19]:
bayes_grid.best_params_

{'vectorizer__max_features': 334}

In [25]:
best_model = bayes_grid.best_estimator_
best_model.fit(X_train, y_train)

print(best_model.score(X_train, y_train))
print(best_model.score(X_test, y_test))

0.9773390172761948
0.9730941704035875


In [26]:
best_model.predict(X_test)

array([0, 0, 1, ..., 0, 0, 1])

## Redes neuronales

In [115]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from keras.metrics import Recall, Precision 

X = dataset_limpio['message']
y = dataset_limpio['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

vectorizer = CountVectorizer(max_features = 334, stop_words = 'english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(X_train_vec.shape)

#print(X_train.toarray().shape)

print(y_train.shape)

ann = Sequential()
ann.add(Dense(500, input_shape=X_train_vec.shape[1:], activation = 'relu'))
ann.add(Dense(300, activation = 'sigmoid'))
ann.add(Dense(100, activation = 'sigmoid'))
ann.add(Dense(2, activation = 'relu'))

ann.summary()

ann.compile(loss='binary_crossentropy', optimizer='adam',
           metrics=['acc', 'mse', 'mae'])

ann.fit(X_train_vec.toarray(), y_train, epochs = 5, batch_size = 100)

(4457, 334)
(4457,)
Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 500)               167500    
_________________________________________________________________
dense_124 (Dense)            (None, 300)               150300    
_________________________________________________________________
dense_125 (Dense)            (None, 100)               30100     
_________________________________________________________________
dense_126 (Dense)            (None, 2)                 202       
Total params: 348,102
Trainable params: 348,102
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
45/45 [==============================] - 0s 2ms/step - loss: 1.9628 - acc: 0.7696 - mse: 0.1385 - mae: 0.1497
Epoch 2/5
45/45 [==============================] - 0s 3ms/step - loss: 2.0661 - acc: 0.8661 - mse: 0.1339

In [116]:
print(X_test_vec.toarray().shape)
print(y_test.shape)

(1115, 334)
(1115,)


In [117]:
prediction = ann.predict(X_test_vec)
prediction

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [118]:
print(y_test.shape)
print(prediction.shape)

(1115,)
(1115, 2)


In [119]:
from sklearn.metrics import roc_auc_score

print('Testing ROC_AUC:', roc_auc_score(y_test, prediction))

ValueError: y should be a 1d array, got an array of shape (1115, 2) instead.